In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv
/kaggle/input/twitter-and-reddit-sentimental-analysis-dataset/Reddit_Data.csv


In [2]:
#training set
path='../input/twitter-and-reddit-sentimental-analysis-dataset/Twitter_Data.csv'
df=pd.read_csv(path)
df.index.name = "index"
df.head()


,clean_text,category
index,,
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [3]:
import nltk
example=df['clean_text'][40]
print(example)

impressive godrej tata complimenting our hoping gets second term 


In [4]:
tokens=nltk.word_tokenize(example)

In [5]:
tagged=nltk.pos_tag(tokens)


In [6]:
entities=nltk.chunk.ne_chunk(tagged)
entities.pprint

<bound method Tree.pprint of Tree('S', [('impressive', 'JJ'), ('godrej', 'NN'), ('tata', 'NN'), ('complimenting', 'VBG'), ('our', 'PRP$'), ('hoping', 'VBG'), ('gets', 'VBZ'), ('second', 'JJ'), ('term', 'NN')])>

In [7]:
#VADER
from nltk.sentiment import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()

/usr/local/lib/python3.10/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [8]:
sia.polarity_scores(example)

{'neg': 0.0, 'neu': 0.39, 'pos': 0.61, 'compound': 0.8555}

In [9]:
df['clean_text'] = df['clean_text'].astype(str)
results = {}
for i, row in df.iterrows():  # Iterate through rows
    tweet = row['clean_text']  # Use row['column_name'] instead of df[column][i]
    results[i] = sia.polarity_scores(tweet)  # Compute sentiment score


    

In [10]:
vaders= pd.DataFrame(results).T
vaders.index.name = "index"
vaders.head()

vaders = vaders.merge(df, how='left', left_index=True, right_index=True)



In [11]:

vaders=vaders.rename(columns={'clean_text':'Tweet'})

In [12]:
vaders


,neg,neu,pos,compound,Tweet,category
index,,,,,,
0,0.065,0.781,0.154,0.5267,when modi promised “minimum government maximum...,-1.0
1,0.184,0.816,0.000,-0.4019,talk all the nonsense and continue all the dra...,0.0
2,0.000,0.772,0.228,0.7096,what did just say vote for modi welcome bjp t...,1.0
3,0.187,0.655,0.158,-0.0713,asking his supporters prefix chowkidar their n...,1.0
4,0.000,0.808,0.192,0.4754,answer who among these the most powerful world...,1.0
...,...,...,...,...,...,...
162975,0.081,0.919,0.000,-0.1280,why these 456 crores paid neerav modi not reco...,-1.0
162976,0.398,0.491,0.111,-0.9571,dear rss terrorist payal gawar what about modi...,-1.0
162977,0.000,1.000,0.000,0.0000,did you cover her interaction forum where she ...,0.0
